# WAL Writing

## When To Flush

- One running transaction has committed or aborted
- The WAL buffer has been filled up with tuples (The WAL buffer size is set to the parameter `wal_buffers`)
- A WAL writer process writes periodically

If any of the above occurs, all WAL records on the WAL buffer are written into a WAL segment file regardless of whether their transactions have been committed or not!


## WAL Writer

Flushes WAL records from WAL buffer to disk by:
- Size (`wal_writer_flush_after` - default is `1MB`)
- Time (`wal_writer_delay` - default is `200ms`)

## Batching WAL Writes

- Async commits (`synchronous_commit=off`, discussed in detail below)
- Commit delay (`commit_delay` & `commit_siblings`) - wait for other active transactions to create ready to flush WAL buffers which results in potential latency but less I/O

### Async Commits

**Idea:** don't wait for WAL records flush to disk to return a transaction success to a client. 

Basically remove the commit / rollback of a transaction as a trigger to WAL disk flush.

Configuration: `synchronous_commit`

```{note}
On replications there are several options too, discussed in detail on replication module.
```

Advantages:
- Can be turned on/off on a transaction level
- Lower latency for transactions
- Potential batching of WAL records flush to disk

Disadvantages:
- Potential risk window of `wal_writer_delay * 3` (because the WAL Writer tries internally to fill up a full page of WAL records) of transactions to be lost on DB crash

```{note}
Async commits are not creating a corruption risk! Since logical commits are flushed sequentially and atomically, only the unwritten transactions are at risk
```

```{note}
If the application that uses the DB is counting on fully sync consistency, for example for a multi-db transaction, it should use a sync commit 
```

#### Recovery Process With WAL

<img src="./helpers/WAL - Recovery.png" alt="drawing" width="900"/>